In [52]:
library(tidyverse)
library(lme4)
library(nlme)
library(car)

FORMULAE IN R: ANOVA AND OTHER MODELS, MIXED AND FIXED  
http://conjugateprior.org/2013/01/formulae-in-r-anova/

Why do lme and aov return different results for repeated measures ANOVA in R?  
https://stats.stackexchange.com/questions/14088/why-do-lme-and-aov-return-different-results-for-repeated-measures-anova-in-r?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa

gls() vs. lme() in the nlme package  
https://stackoverflow.com/questions/1395102/gls-vs-lme-in-the-nlme-package?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa

In [3]:
dat_pearl <- tribble(
    ~market_value, ~coat, ~batch, ~rep,
72.0, 1, 1, 1, 74.6, 1, 1, 2, 67.4, 1, 1, 3,
72.8, 1, 1, 4, 72.1, 1, 2, 1, 76.9, 1, 2, 2,
74.8, 1, 2, 3, 73.3, 1, 2, 4, 75.2, 1, 3, 1,
73.8, 1, 3, 2, 75.7, 1, 3, 3, 77.8, 1, 3, 4,
70.4, 1, 4, 1, 68.1, 1, 4, 2, 72.4, 1, 4, 3,
72.4, 1, 4, 4, 76.9, 2, 1, 1, 78.1, 2, 1, 2,
72.9, 2, 1, 3, 74.2, 2, 1, 4, 80.3, 2, 2, 1,
79.3, 2, 2, 2, 76.6, 2, 2, 3, 77.2, 2, 2, 4,
80.2, 2, 3, 1, 76.6, 2, 3, 2, 77.3, 2, 3, 3,
79.9, 2, 3, 4, 74.3, 2, 4, 1, 77.6, 2, 4, 2,
74.4, 2, 4, 3, 72.9, 2, 4, 4, 76.3, 3, 1, 1,
74.1, 3, 1, 2, 77.1, 3, 1, 3, 75.0, 3, 1, 4,
80.9, 3, 2, 1, 73.7, 3, 2, 2, 78.6, 3, 2, 3,
80.2, 3, 2, 4, 79.2, 3, 3, 1, 78.0, 3, 3, 2,
77.6, 3, 3, 3, 81.2, 3, 3, 4, 71.6, 3, 4, 1,
77.7, 3, 4, 2, 75.2, 3, 4, 3, 74.4, 3, 4, 4)

dat_pearl$coat  <- factor(dat_pearl$coat)
dat_pearl$batch <- factor(dat_pearl$batch)

head(dat_pearl, 2)
tail(dat_pearl, 2)

market_value,coat,batch,rep
72.0,1,1,1
74.6,1,1,2


market_value,coat,batch,rep
75.2,3,4,3
74.4,3,4,4


# ANOVA: Fixed Effect 

In [18]:
res <- aov(
    market_value ~ coat + batch + coat:batch, 
    data = dat_pearl)
summary(res)

            Df Sum Sq Mean Sq F value   Pr(>F)    
coat         2 150.39   75.19  15.591 1.33e-05 ***
batch        3 152.85   50.95  10.564 3.98e-05 ***
coat:batch   6   1.85    0.31   0.064    0.999    
Residuals   36 173.62    4.82                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [20]:
res <- aov(
    market_value ~ coat + batch + coat*batch, 
    data = dat_pearl)
summary(res)

            Df Sum Sq Mean Sq F value   Pr(>F)    
coat         2 150.39   75.19  15.591 1.33e-05 ***
batch        3 152.85   50.95  10.564 3.98e-05 ***
coat:batch   6   1.85    0.31   0.064    0.999    
Residuals   36 173.62    4.82                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

```
            Df Sum Sq Mean Sq F value   Pr(>F)    
Model       11 305.09   27.74   5.754
coat         2 150.39   75.19  15.591 1.33e-05 ***
batch        3 152.85   50.95  10.564 3.98e-05 ***
coat:batch   6   1.85    0.31   0.064    0.999    
Residuals   36 173.62    4.82                     
```

In [48]:
cat(150.39 + 152.85 + 1.85, "\n")
cat(305.09 / 11, "\n")
cat(305.09 / 11 / 4.82)

305.09 
27.73545 
5.754244

# Mean in each group

In [23]:
print(model.tables(res,"means"),digits=3)

Tables of means
Grand mean
         
75.60833 

 coat 
coat
   1    2    3 
73.1 76.8 76.9 

 batch 
batch
   1    2    3    4 
74.3 77.0 77.7 73.5 

 coat:batch 
    batch
coat 1    2    3    4   
   1 71.7 74.3 75.6 70.8
   2 75.5 78.4 78.5 74.8
   3 75.6 78.4 79.0 74.7


In [30]:
dat_pearl %>% group_by(coat, batch) %>% summarize(Mean = mean(market_value)) %>% spread(batch, Mean) %>% print(digit = 3)

# A tibble: 3 x 5
# Groups:   coat [3]
  coat    `1`   `2`   `3`   `4`
  <fct> <dbl> <dbl> <dbl> <dbl>
1 1      71.7  74.3  75.6  70.8
2 2      75.5  78.4  78.5  74.8
3 3      75.6  78.4  79.0  74.7


In [24]:
head(dat_pearl)

market_value,coat,batch,rep
72.0,1,1,1
74.6,1,1,2
67.4,1,1,3
72.8,1,1,4
72.1,1,2,1
76.9,1,2,2


# ANOVA: Random Effect

In [12]:
res <- aov(
    market_value ~ coat + Error(batch + coat * batch), 
    data = dat_pearl)
summary(res)


Error: batch
          Df Sum Sq Mean Sq F value Pr(>F)
Residuals  3  152.8   50.95               

Error: coat
     Df Sum Sq Mean Sq
coat  2  150.4   75.19

Error: batch:coat
          Df Sum Sq Mean Sq F value Pr(>F)
Residuals  6  1.852  0.3087               

Error: Within
          Df Sum Sq Mean Sq F value Pr(>F)
Residuals 36  173.6   4.823               

In [60]:
res <- aov(
    market_value ~ coat + Error(batch/coat) + coat * batch, 
    data = dat_pearl)
#summary(res)$`Error: batch`
summary(res)


Error: batch
      Df Sum Sq Mean Sq
batch  3  152.8   50.95

Error: batch:coat
           Df Sum Sq Mean Sq
coat        2 150.39   75.19
coat:batch  6   1.85    0.31

Error: Within
          Df Sum Sq Mean Sq F value Pr(>F)
Residuals 36  173.6   4.823               

In [61]:
res <- lmer(
    market_value ~ coat + (1 | batch) + (1 | coat : batch), 
    data = dat_pearl)
summary(res)

Linear mixed model fit by REML ['lmerMod']
Formula: market_value ~ coat + (1 | batch) + (1 | coat:batch)
   Data: dat_pearl

REML criterion at convergence: 207.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.1991 -0.6281  0.1033  0.6555  1.3638 

Random effects:
 Groups     Name        Variance Std.Dev.
 coat:batch (Intercept) 0.000    0.000   
 batch      (Intercept) 3.898    1.974   
 Residual               4.178    2.044   
Number of obs: 48, groups:  coat:batch, 12; batch, 4

Fixed effects:
            Estimate Std. Error t value
(Intercept)  73.1062     1.1116  65.769
coat2         3.6875     0.7227   5.103
coat3         3.8188     0.7227   5.284

Correlation of Fixed Effects:
      (Intr) coat2 
coat2 -0.325       
coat3 -0.325  0.500

In [ ]:
class coat batch;
model market_value=coat batch coat*batch;
random batch coat*batch/test;
means coat/tukey lines e=coat*batch;
